In [1]:
! nvidia-smi

Wed Jun 15 16:03:39 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 512.95       Driver Version: 512.95       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:09:00.0  On |                  N/A |
| 18%   38C    P8    10W / 185W |    834MiB /  8192MiB |     24%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [6]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling ~\tensorflow_datasets\mnist\3.0.1.incomplete9K8971\mnist-train.tfrecord*...:   0%|          | 0/6000…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling ~\tensorflow_datasets\mnist\3.0.1.incomplete9K8971\mnist-test.tfrecord*...:   0%|          | 0/10000…

Dataset mnist downloaded and prepared to ~\tensorflow_datasets\mnist\3.0.1. Subsequent calls will reuse this data.


In [3]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

In [4]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [7]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

with tf.device('/GPU:0'):
  model.fit(
      ds_train,
      epochs=6,
      validation_data=ds_test,
  )

Epoch 1/6
469/469 [==============================] - 3s 5ms/step - loss: 0.3560 - sparse_categorical_accuracy: 0.9029 - val_loss: 0.2032 - val_sparse_categorical_accuracy: 0.9428
Epoch 2/6
469/469 [==============================] - 2s 4ms/step - loss: 0.1672 - sparse_categorical_accuracy: 0.9528 - val_loss: 0.1412 - val_sparse_categorical_accuracy: 0.9579
Epoch 3/6
469/469 [==============================] - 2s 4ms/step - loss: 0.1199 - sparse_categorical_accuracy: 0.9659 - val_loss: 0.1203 - val_sparse_categorical_accuracy: 0.9650
Epoch 4/6
469/469 [==============================] - 2s 4ms/step - loss: 0.0939 - sparse_categorical_accuracy: 0.9733 - val_loss: 0.0976 - val_sparse_categorical_accuracy: 0.9711
Epoch 5/6
469/469 [==============================] - 2s 4ms/step - loss: 0.0753 - sparse_categorical_accuracy: 0.9781 - val_loss: 0.0857 - val_sparse_categorical_accuracy: 0.9743
Epoch 6/6
469/469 [==============================] - 2s 4ms/step - loss: 0.0616 - sparse_categorical_accu